# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

# Para rodar no segundo plano
os.environ['MOZ_HEADLESS'] = '1'

# Para o Chrome
nav = webdriver.Chrome()

# Entrar no google
nav.get("https://www.google.com/")

# DOLAR - Utiliza xpath para achar elemento da barra de busca e escreve
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do dólar")
# Enter
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# EURO - Utiliza xpath para achar elemento da barra de busca e escreve
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do euro")
# Enter
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# OURO - Utiliza xpath para achar elemento da barra de busca e escreve
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0]
nav.find_element_by_xpath('/html/body/div[15]/div[2]/div/table[2]/tbody/tr[2]/td[2]/a/img').click()
# Mudar de aba
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
ouro = nav.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/input[2]').get_attribute("value")
ouro = ouro.replace(",", ".")

nav.quit()

print("dolar: ", dolar)
print("euro: ", euro)
print("ouro: ", ouro)

dolar:  4.9353
euro:  5.884802367000001
ouro 282.71


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [19]:
import pandas as pd 

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [22]:
# Na coluna "Moeda" da tabela onde for "Dólar" altera o valor de "Cotação na mesma linha"
tabela.loc[tabela['Moeda']=="Dólar", "Cotação"] = float(dolar)
# O mesmo para euro e ouro
tabela.loc[tabela['Moeda']=="Euro", "Cotação"] = float(euro)
tabela.loc[tabela['Moeda']=="Ouro", "Cotação"] = float(ouro)

tabela['Preço Base Reais'] = tabela['Preço Base Original'] * tabela['Cotação'] 
tabela['Preço Final'] = tabela['Preço Base Reais'] * tabela['Margem']

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.935300,4935.250647,1.40,6909.350906
1,Carro Renault,4500.00,Euro,5.884802,26481.610652,2.00,52963.221303
2,Notebook Dell,899.99,Dólar,4.935300,4441.720647,1.70,7550.925100
3,IPhone,799.00,Dólar,4.935300,3943.304700,1.70,6703.617990
4,Carro Fiat,3000.00,Euro,5.884802,17654.407101,1.90,33543.373492
5,Celular Xiaomi,480.48,Dólar,4.935300,2371.312944,2.00,4742.625888
6,Joia 20g,20.00,Ouro,282.710000,5654.200000,1.15,6502.330000


### Agora vamos exportar a nova base de preços atualizada

In [23]:
# Tabela exportada sem índices
tabela.to_excel("Produtos Atualizado.xlsx", index=False)